![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

In [1]:
# QuantBook Analysis Tool 
# For more information see [https://www.quantconnect.com/docs/v2/our-platform/research/getting-started]
qb = QuantBook()
spy = qb.AddEquity("SPY")
# Locally Lean installs free sample data, to download more data please visit https://www.quantconnect.com/docs/v2/lean-cli/datasets/downloading-data 
qb.SetStartDate(2013, 10, 11)
history = qb.History(qb.Securities.Keys, 360, Resolution.Daily)

# Indicator Analysis
bbdf = qb.Indicator(BollingerBands(30, 2), spy.Symbol, 360, Resolution.Daily)
bbdf.drop('standarddeviation', axis=1).plot()

NameError: name 'QuantBook' is not defined

In [3]:
from datetime import timedelta
from QuantConnect.Data.Consolidators import TradeBarConsolidator
from QuantConnect.Indicators import Maximum

class StockTrackingAlgorithm(QCAlgorithm):
    def Initialize(self):
        self.SetStartDate(2023, 1, 1)
        self.SetCash(100000)
        self.symbols = ["TSLA", "MSFT", "NVDA", "AAPL", "GOOGL"]
        self.max_10_day_high = {}
        self.entry_prices = {}
        
        # Create a 10-day maximum indicator for each stock
        for symbol in self.symbols:
            equity = self.AddEquity(symbol, Resolution.Daily)
            self.max_10_day_high[symbol] = Maximum(10)
            consolidator = TradeBarConsolidator(timedelta(days=1))
            consolidator.DataConsolidated += self.OnDataConsolidated
            self.RegisterIndicator(symbol, self.max_10_day_high[symbol], consolidator)
            self.entry_prices[symbol] = None

    def OnDataConsolidated(self, sender, bar):
        symbol = bar.Symbol.Value
        max_10_day_high = self.max_10_day_high[symbol]

        # Entry condition: stock falls by 10% from its 10-day high
        if self.entry_prices[symbol] is None and bar.Close < max_10_day_high.Current.Value * 0.9:
            self.SetHoldings(symbol, 0.2)
            self.entry_prices[symbol] = bar.Close

        # Exit condition: 5% trailing take profit
        elif self.entry_prices[symbol] and bar.Close >= self.entry_prices[symbol] * 1.05:
            self.Liquidate(symbol)
            self.entry_prices[symbol] = None

    def OnData(self, data):
        pass

ModuleNotFoundError: No module named 'QuantConnect'